In [1]:
import findspark
findspark.init('/home/ronit/spark-3.0.0-preview2-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [5]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [6]:
my_log_reg_model = LogisticRegression()

In [7]:
fitted_log_reg = my_log_reg_model.fit(my_data)

In [8]:
log_summary = fitted_log_reg.summary

In [9]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [13]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111048...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126969...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170264...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200596...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695572...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102604...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215612...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [18]:
lr_train , lr_test = my_data.randomSplit([0.7,0.3])

In [19]:
final_model = LogisticRegression()

In [21]:
fit_final = final_model.fit(lr_train)

In [22]:
predictions_and_labels = fit_final.evaluate(lr_test)

In [23]:
predictions_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[98,99,100,1...|[31.0822618342253...|[0.99999999999996...|       0.0|
|  0.0|(692,[100,101,102...|[12.8850535259216...|[0.99999746433863...|       0.0|
|  0.0|(692,[122,123,124...|[18.9788791312060...|[0.99999999427760...|       0.0|
|  0.0|(692,[122,123,148...|[21.1779690212311...|[0.99999999936536...|       0.0|
|  0.0|(692,[124,125,126...|[29.9348975875556...|[0.99999999999990...|       0.0|
|  0.0|(692,[124,125,126...|[29.6006113450676...|[0.99999999999986...|       0.0|
|  0.0|(692,[124,125,126...|[22.9471090739664...|[0.99999999989180...|       0.0|
|  0.0|(692,[125,126,127...|[22.6249813589783...|[0.99999999985068...|       0.0|
|  0.0|(692,[126,127,128...|[33.3219280900263...|[0.99999999999999...|       0.0|
|  0.0|(692,[126

In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [26]:
my_eval = BinaryClassificationEvaluator()

In [28]:
my_final_roc = my_eval.evaluate(predictions_and_labels.predictions)

In [29]:
my_final_roc

1.0

In [30]:
spark = SparkSession.builder.appName('titanis').getOrCreate()

In [31]:
df = spark.read.csv('titanic.csv', header=True, inferSchema=True)

In [53]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [54]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [55]:
my_cols = df.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [56]:
my_final_data = my_cols.na.drop()

In [57]:
from pyspark.ml.feature import VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer

In [58]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')

In [59]:
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [60]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [61]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','Age','SibSp','Parch','Fare','EmbarkVec'], outputCol='features')

In [62]:
from pyspark.ml import Pipeline

In [63]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [64]:
pipline = Pipeline(stages=[gender_indexer,embark_indexer,gender_encoder,embark_encoder,assembler,log_reg_titanic])

In [65]:
train_data , test_data = my_final_data.randomSplit([0.7,0.3])

In [66]:
fit_model = pipline.fit(train_data)

In [67]:
results = fit_model.transform(test_data)

In [68]:
type(results)

pyspark.sql.dataframe.DataFrame

In [73]:
results.columns

['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked',
 'SexIndex',
 'EmbarkIndex',
 'SexVec',
 'EmbarkVec',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [75]:
from pyspark.sql.functions import format_number

In [77]:
results.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- SexIndex: double (nullable = false)
 |-- EmbarkIndex: double (nullable = false)
 |-- SexVec: vector (nullable = true)
 |-- EmbarkVec: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [79]:
results.head(1)

[Row(Survived=0, Pclass=1, Sex='female', Age=25.0, SibSp=1, Parch=2, Fare=151.55, Embarked='S', SexIndex=1.0, EmbarkIndex=0.0, SexVec=SparseVector(1, {}), EmbarkVec=SparseVector(2, {0: 1.0}), features=DenseVector([1.0, 0.0, 25.0, 1.0, 2.0, 151.55, 1.0, 0.0]), rawPrediction=DenseVector([-2.5908, 2.5908]), probability=DenseVector([0.0697, 0.9303]), prediction=1.0)]

In [78]:
results.select(['Survived','rawPrediction',
 'probability',
 'prediction']).show()

+--------+--------------------+--------------------+----------+
|Survived|       rawPrediction|         probability|prediction|
+--------+--------------------+--------------------+----------+
|       0|[-2.5907590860840...|[0.06973552299360...|       1.0|
|       0|[-2.2253701685183...|[0.09749526108359...|       1.0|
|       0|[-1.0716849094968...|[0.25508279305137...|       1.0|
|       0|[-0.3907166795746...|[0.40354478659742...|       1.0|
|       0|[-0.1179799581841...|[0.47053917529469...|       1.0|
|       0|[-0.2213546173170...|[0.44488620032869...|       1.0|
|       0|[-0.5736700204815...|[0.36039041877196...|       1.0|
|       0|[0.14052786077582...|[0.53507426340397...|       0.0|
|       0|[0.30336344643925...|[0.57526453265558...|       0.0|
|       0|[0.40261451037812...|[0.59931566193094...|       0.0|
|       0|[1.55138278168284...|[0.82511335874851...|       0.0|
|       0|[0.57193716477415...|[0.63921004554915...|       0.0|
|       0|[0.55275910166633...|[0.634775

In [70]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [71]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [81]:
auc = my_eval.evaluate(results)

In [82]:
auc

0.7978838582677166